In [1]:
# Cell 1: Install dependencies (once)
# !conda install -c conda-forge \
#     dask distributed scikit-learn pandas pyarrow -y


In [2]:
# Cell 2: Imports & Dask client (for delayed)
import os, random, numpy as np, pandas as pd
from dask import delayed
from dask.distributed import Client

# Scikit-learn
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer
from sklearn.cluster import MiniBatchKMeans

# Reproducibility
random.seed(42)
np.random.seed(42)

# Launch a local Dask client (optional—you can still .compute() sequentially)
client = Client()
print(client)


<Client: 'tcp://127.0.0.1:40749' processes=4 threads=16, memory=30.49 GiB>


In [3]:
GAME_THEMES = {

    391540 :
    {# Branching narrative and emotional beats
    'story': [
        'story', 'plot', 'narrative', 'pacifist', 'genocide', 'neutral',
        'branching', 'ending', 'choice', 'consequence', 'moral',
        'friendship', 'redemption', 'hope', 'love', 'loss', 'memory',
        'determination'
    ],

    # Cast of monsters, humans, and the player avatar
    'characters': [
        'character', 'frisk', 'chara', 'sans', 'papyrus', 'toriel',
        'asgore', 'alphys', 'undyne', 'mettaton', 'flowey', 'asriel',
        'napstablook', 'monster kid', 'temmie', 'burgerpants',
        'npc', 'relationship', 'backstory'
    ],

    # Core RPG systems and moment‑to‑moment interaction
    'gameplay': [
        'gameplay', 'rpg', 'adventure', 'exploration', 'puzzle',
        'bullet hell', 'dodging', 'timing', 'menu', 'save', 'load',
        'checkpoint', 'inventory', 'quest', 'side quest', 'boss fight'
    ],

    # Turn‑based battles, ACT options, and mercy mechanics
    'combat': [
        'combat', 'battle', 'fight', 'act', 'mercy', 'spare', 'talk',
        'attack', 'defense', 'hp', 'exp', 'lv', 'soul', 'heart',
        'pattern', 'special attack', 'blue soul', 'green soul',
        'orange soul', 'purple soul', 'yellow soul'
    ],

    # Iconic chiptune OST by Toby Fox
    'music': [
        'music', 'soundtrack', 'ost', 'song', 'track', 'theme', 'melody',
        'motif', 'chiptune', 'chip‑tune', 'Megalovania', 'Hopes and Dreams',
        'His Theme', 'Undertale OST', 'piano', '8‑bit', 'orchestral'
    ],

    # Retro pixel art presentation and UI
    'visuals': [
        'visuals', 'graphics', 'sprite', 'pixel art', 'retro', 'color',
        'palette', 'animation', 'effect', 'lighting', 'ui', 'hud',
        'text box', 'portrait', 'scanline', 'CRT'
    ],

    # Signature humor, meta commentary, and fourth‑wall breaks
    'humor_dialogue': [
        'humor', 'dialogue', 'pun', 'joke', 'quirky', 'whimsical',
        'meta', 'fourth wall', 'narrator', 'sarcasm', 'flavor text',
        'reference', 'running gag', 'comic timing'
    ],

    # Moral philosophy, player agency, and in‑game judgment
    'morality': [
        'morality', 'ethics', 'choice', 'consequence', 'violence',
        'mercy', 'guilt', 'atonement', 'judgment hall', 'sins',
        'player agency', 'self‑reflection'
    ],

    # Hidden events, ARG elements, and file manipulation
    'secrets_meta': [
        'secret', 'hidden', 'easter egg', 'gaster', 'fun value',
        'mystery door', 'alternate route', 'hard mode', 'reset',
        'true lab', 'file delete', 'save file', 'corruption',
        'exe', 'meta narrative'
    ],

    # Regions, set pieces, and environmental storytelling
    'warp': [
        'world', 'setting', 'underground', 'ruins', 'snowdin',
        'waterfall', 'hotland', 'core', 'new home', 'castle',
        'true lab', 'temmie village', 'river person', 'map',
        'environment', 'atmosphere'
    ]
    },
    10: {
    # Core moment‑to‑moment play
    'gameplay': [
        'gameplay', 'mechanics', 'tactical shooter', 'precision', 'aim',
        'headshot', 'spray‑control', 'burst‑fire', 'recoil', 'crosshair',
        'peek', 'counter‑strafe', 'movement', 'jump‑peek', 'clutch',
        'bomb plant', 'defuse', 'round', 'eco‑round', 'overtime'
    ],

    # Weapons, grenades, recoil patterns
    'weapons': [
        'weapon', 'gun', 'rifle', 'ak', 'm4', 'awp', 'pistol', 'deagle',
        'smg', 'shotgun', 'sniper', 'knife', 'grenade', 'flashbang',
        'smoke', 'molotov', 'he‑nade', 'incendiary', 'zeus', 'spray',
        'pull‑out time', 'reload'
    ],

    # Map design and call‑outs
    'maps': [
        'map', 'layout', 'bombsite', 'call‑out', 'rotation', 'angles',
        'cover', 'line‑up', 'utility spot', 'choke‑point', 'dust2',
        'mirage', 'inferno', 'nuke', 'overpass', 'ancient', 'vertigo',
        'office', 'train', 'cache'
    ],

    # Ranked play, esport angle, organised competition
    'competitive': [
        'competitive', 'matchmaking', 'rank', 'elo', 'premier', 'global elite',
        'silver', 'faceit', 'esport', 'tournament', 'major', 'league',
        'teamplay', 'strat', 'timeout', 'coach', 'demo review', 'practice'
    ],

    # Game economy, skins, trading and cases
    'economy & skins': [
        'economy', 'money', 'buy', 'force', 'save', 'full‑buy', 'bonus‑loss',
        'skin', 'knife skin', 'case', 'capsule', 'stattrak', 'souvenir',
        'sticker', 'trade‑up', 'market', 'auction', 'rarity', 'float value',
        'pattern', 'lootbox'
    ],

    # Anti‑cheat and integrity concerns
    'anti_cheat': [
        'cheater', 'cheat', 'hacker', 'hack', 'wallhack', 'aimbot', 'spinbot',
        'vac', 'vac ban', 'prime', 'overwatch', 'smurf', 'rage', 'backtracking',
        'triggerbot', 'report', 'banwave', 'trust factor'
    ],

    # Performance, networking and technical stability
    'performance': [
        'fps', 'frame rate', 'stutter', 'lag', 'ping', 'tickrate', 'sub‑tick',
        'server', 'hit‑reg', 'netcode', 'desync', 'packet loss', 'freeze',
        'crash', 'memory leak', 'loading time', 'update', 'patch', 'driver'
    ],

    # Visual fidelity and art direction
    'visuals': [
        'visuals', 'graphics', 'shader', 'lighting', 'smoke effect',
        'blood splatter', 'particle', 'texture', 'model', 'animation',
        'ui', 'hud', 'crosshair style', 'ray tracing', 'color', 'resolution',
        'fov', 'viewmodel'
    ],

    # Audio design and voice comms
    'audio': [
        'audio', 'sound', 'footstep', 'sound cue', 'directional',
        'occlusion', 'gunshot', 'reverb', 'bomb beep', 'defuse sound',
        'voice chat', 'callout', 'microphone', 'radio command', 'volume',
        'sound bug', 'muffle', 'mix'
    ],

    # Social experience and community features
    'community': [
        'community', 'friends', 'lobby', 'party', 'team‑mate', 'toxic',
        'grief', 'vote kick', 'chat', 'text chat', 'mute', 'spray logo',
        'workshop', 'community server', 'surf', 'bhop', 'mods', 'plugin',
        'custom map', 'training map'
    ]
},
1794680 :{
    # Moment‑to‑moment play and core loop
    'gameplay': [
        'gameplay', 'rogue‑lite', 'bullet heaven', 'bullet hell',
        'auto‑shooter', 'idle shooter', 'survival', 'run', 'session',
        'wave', 'horde', 'timer', '30‑minute mark', 'stage clear',
        'pickup', 'level‑up', 'reroll', 'skip', 'banish', 'gold rush'
    ],

    # Offensive and defensive gear (base + evolutions + unions)
    'weapons': [
        # base
        'whip', 'magic wand', 'knife', 'axe', 'cross', 'king bible',
        'fire wand', 'garlic', 'lightning ring', 'rune tracer',
        'pentagram', 'song of mana', 'shadow pinion', 'laurel',
        'clock lancet', 'flames of misspell', 'jettonion', 'bracelets',
        # evolutions / unions
        'bloody tear', 'holy wand', 'infinite corridor', 'crimson shroud',
        'heaven sword', 'hellfire', 'thunder loop', 'labora',
        'gorgeous moon', 'mannajja', 'vicious hunger', 'no future',
        'fuwalafuwaloo', 'phieraggi', 'ashes of muspell', 'legionnaire',
        'spellstrom', 'night sword', 'muramasa'
    ],

    # Passive items that unlock evolutions or buffs
    'passive_items': [
        'spinach', 'candelabrador', 'bracer', 'clover', 'duplicator',
        'tiragisú', 'wings', 'empty tome', 'armor', 'pummarola',
        'heart', 'stone mask', 'skull o’maniac', 'crown', 'curse',
        'torrona’s box', 'golden egg', 'super candybox II turbo'
    ],

    # Playable cast and coffins
    'characters': [
        'antonio', 'impi', 'pascallina', 'gennaro', 'arcanist',
        'clerici', 'mortaccio', 'porta', 'llama', 'christine', 'pugnala',
        'giovanna', 'poppea', 'concetta', 'bianca', 'oskari',  # DLC
        'queen sigma', 'big trouser', 'red death', 'missingno',
        'secret', 'coffin', 'unlock', 'starting weapon', 'growth'
    ],

    # Random‑draw mutators that reshape a run
    'arcana': [
        'arcana', 'card', 'VIII – Mad Groove', 'VI – Sarabande of Healing',
        'XX – Silent Old Sanctuary', 'XVI – Slash', 'XI – Waltz of Pearls',
        'XXI – Blood Astronomia', 'XVII – Lost & Found Painting',
        'banish card', 'draw', 'deck', 'reroll', 'curse arcana'
    ],

    # Maps, stage modifiers, and set‑piece gimmicks
    'stages': [
        'stage', 'mad forest', 'inlaid library', 'dairy plant',
        'gallo tower', 'cappella magna', 'bone zone', 'moongolow',
        'boss rash', 'il molise', 'green acres', 'endless mode',
        'inverse mode', 'hyper mode', 'limit break', 'tides of the foscari',
        'legacy of the moonspell', 'operation guns', 'astral stair',
        'lake fuma', 'space 54'
    ],

    # Persistent unlocks and long‑term progression
    'meta_progression': [
        'power up', 'power‑ups menu', 'gold', 'merchant',
        'eggs', 'golden egg', 'limit break', 'relic', 'milky way map',
        'grim grimoire', 'arcana unlock', 'jelii crystal',
        'randomazzo', 'collection', 'achievement', 'unlock condition'
    ],

    # Reaper‑spawn, boss waves, cursed runs, endless grinds
    'difficulty_modes': [
        'difficulty', 'hyper', 'inverse', 'endless', 'hurry', 'boss rush',
        'curse', 'skull', 'minute 20 boss', 'death', 'red death',
        'white death', 'green death', 'stalker', 'drowner'
    ],

    # Swarms, minibosses, and on‑screen clutter
    'enemies': [
        'enemy', 'bat', 'giant bat', 'skeleton', 'ghost', 'werewolf',
        'witch', 'mummy', 'reaper', 'plant ring', 'golem',
        'lion head', 'bone dragon', 'valkyrie', 'guardian', 'boss',
        'mob density', 'spawning', 'hp scaling'
    ],

    # Pixel‑art spectacle and UI readability
    'visuals': [
        'visuals', 'pixel art', 'retro', 'sprite', 'color burst',
        'particle', 'flash', 'damage numbers', 'outline', 'shader',
        'vfx', 'screen shake', 'hud', 'ui', 'stage lighting', 'night cycle'
    ],

    # Chip‑tune bangers and crunchy SFX
    'audio': [
        'audio', 'music', 'soundtrack', 'ost', 'track', 'loop',
        'banger', 'chiptune', 'sfx', 'pickup sound', 'level‑up jingle',
        'healing ping', 'coin chime', 'death roar', 'mute', 'volume'
    ],

    # Technical polish and device performance
    'performance': [
        'fps', 'frame rate', 'lag', 'stutter', 'slowdown', 'optimization',
        'mobile port', 'steam deck', 'controller support', 'rumble',
        'graphics setting', 'update patch', 'save file', 'cloud save'
    ],

    # Community, mods, patches, crossovers
    'community': [
        'community', 'patch', 'update', 'dlc', 'mod', 'workshop',
        'guide', 'wiki', 'speedrun', 'challenge', 'leaderboard',
        'theorycraft', 'build', 'tier list', 'crossover', 'among us',
        'collab', 'feedback', 'balancing'
    ]
},
1966720: {
    # Core co‑op loop: landing, looting, leaving
    'gameplay': [
        'gameplay', 'co‑op', 'multiplayer', 'first‑person', 'salvage',
        'scrap collecting', 'quota', 'profit quota', 'deadline', 'shift',
        'timer', 'hazard level', 'weather', 'eclipse', 'fog', 'storm',
        'launch', 'land', 'extraction', 'escape', 'weight limit',
        'door codes', 'lever', 'ship recall'
    ],

    # Toolkit you buy from the Company Store
    'equipment': [
        'equipment', 'flashlight', 'pro flashlight', 'lantern', 'boombox',
        'walkie‑talkie', 'radar‑booster', 'lockpicker', 'stun grenade',
        'flashbang', 'zap gun', 'shovel', 'yield sign', 'stop sign',
        'shotgun', 'shell', 'jetpack', 'inverse teleporter', 'extendo grip',
        'clown horn', 'signal translator', 'spray paint', 'bunk beds',
        'terminal decoration'
    ],

    # Hostile entities roaming interiors and exteriors
    'enemies': [
        'enemy', 'entity', 'Bracken', 'Coil‑Head', 'Masked', 'Jester',
        'Thumper', 'Circuit Bee', 'Eyeless Dog', 'Forest Keeper',
        'Maneater', 'Barber', 'Butler', 'Old Bird', 'Tulip Snake',
        'Ghost Girl', 'Baboon Hawk', 'Earth Leviathan', 'Manticoil',
        'Snare Flea', 'Bunker Spider'
    ],

    # Passive dangers, traps and environmental threats
    'hazards': [
        'hazard', 'turret', 'landmine', 'spike trap', 'sentry drone',
        'electrical trap', 'pitfall', 'fire exit alarm', 'toxic gas',
        'sandstorm', 'flash storm', 'flood rain', 'heat wave',
        'eclipse darkness'
    ],

    # 15 explorable moons plus Company HQ
    'moons': [
        'Gordion', 'Experimentation', 'Assurance', 'Vow', 'Offense',
        'March', 'Adamance', 'Rend', 'Dine', 'Titan',
        'Embrion', 'Artifice', 'Liquidation', 'Challenge Moon',
        'Weekly Challenge', 'The Company'
    ],

    # Scrap objects you haul back for credits
    'scrap': [
        'scrap', 'large axle', 'gold bar', 'v‑type engine', 'cash register',
        'rainbow fish', 'teeth', 'brass bell', 'antique vase',
        'air horn', 'ancient coin', 'music box', 'toy robot',
        'applicator', 'apparatus', 'gift', 'painting', 'ring', 'mask'
    ],

    # In‑run information systems
    'terminal_ops': [
        'terminal', 'scan', 'ping', 'view monitor', 'door', 'unlock',
        'open', 'close', 'overload', 'disable turret', 'disable mine',
        'store', 'confirm', 'cancel', 'ship status', 'credits balance'
    ],

    # Long‑term progression & Company economy
    'economy': [
        'company credits', 'store', 'buy', 'sell', 'price', 'discount',
        'tax', 'profit quota', 'deadline day', 'quota meter',
        'power‑cell upgrade', 'engine upgrade', 'radar', 'decor rotation'
    ],

    # Team coordination & VOIP
    'communication': [
        'proximity voice', 'radio', 'walkie', 'callout', 'code word',
        'map call‑outs', 'lobby chat', 'text chat', 'signal sound',
        'horn', 'boombox beat', 'mission planning'
    ],

    # Presentation layers
    'audio_visuals': [
        'audio', 'footstep', 'monster cue', 'music stinger',
        'heartbeat', 'proximity static', 'flashlight beam', 'CRT UI',
        'pixel‑low‑poly', 'shader', 'shadow', 'rain effect',
        'screen shake', 'vfx clutter'
    ],

    # Stability & tech footprint
    'performance': [
        'fps', 'optimization', 'lag spike', 'desync', 'netcode',
        'rubber‑band', 'save bug', 'crash', 'hotfix', 'patch',
        'steam deck', 'ultrawide', 'controller'
    ],

    # Fan ecosystem & post‑launch support
    'community': [
        'community', 'update', 'patch notes', 'version 55',
        'mod', 'workshop', 'steam guide', 'tier list',
        'challenge seed', 'speedrun', 'clip', 'meme',
        'feedback', 'balancing', 'roadmap'
    ]
}
}
n_themes = len(GAME_THEMES)


In [4]:
# Cell 4: Global hashing & kmeans params
N_FEATURES   = 2**18     # ~260 K dims
BATCH_SIZE   = 5000      # pages per partial_fit
SAMPLE_PER_C = 5000      # TF-IDF docs per cluster

# These live here so we reuse the same settings in every task


In [5]:
# Cell 5: Per-game analysis function (fixed seeding)

@delayed
def analyse_one_game(appid, themes):
    # 1) Load only this game's reviews
    path = f'/home/rgmatr1x/Documents/projects/steamETL/parquet_output_theme_combo/{appid}.parquet'
    if not os.path.exists(path):
        print(f"⚠️ Missing file for appid {appid}")
        return None

    df = pd.read_parquet(path, columns=['review','votes_up','voted_up','review_language'])
    df = df[df.review_language=='english'].dropna(subset=['review'])
    df['review'] = df['review'].astype(str)
    n = len(df)
    if n == 0:
        print(f"⚠️ No English reviews for {appid}")
        return None

    # 2) Build seed centroids from your theme keywords
    vec = HashingVectorizer(
        n_features=N_FEATURES,
        alternate_sign=False,
        stop_words='english',
        dtype=np.float32
    )
    pseudo_docs    = [" ".join(ws) for ws in themes.values()]
    pseudo_sparse  = vec.transform(pseudo_docs)  # SciPy CSR: (n_themes, n_features)
    init_centroids = np.vstack([row.toarray().ravel() for row in pseudo_sparse])

    # 3) Configure MiniBatchKMeans with custom init
    mbkm = MiniBatchKMeans(
        n_clusters=   len(themes),
        init=         init_centroids,
        n_init=       1,
        random_state= 42,
        batch_size=   BATCH_SIZE
    )

    # 4) Partial-fit in batches
    labels = np.empty(n, dtype=int)
    for start in range(0, n, BATCH_SIZE):
        end = min(n, start + BATCH_SIZE)
        Xb  = vec.transform(df['review'].iloc[start:end])
        mbkm.partial_fit(Xb)
        labels[start:end] = mbkm.predict(Xb)

    df['topic_id'] = labels

    # 5) Sample per-cluster and fit TF-IDF for keywords
    all_samples  = []
    cluster_docs = {}
    for tid in range(len(themes)):
        docs = df.loc[df.topic_id==tid, 'review']
        sampled = docs.sample(min(len(docs), SAMPLE_PER_C), random_state=42).tolist()
        cluster_docs[tid] = sampled
        all_samples.extend(sampled)

    tfv   = TfidfVectorizer(max_features=2000,
                             stop_words='english',
                             ngram_range=(1,2))
    tfidf = tfv.fit_transform(all_samples)

    keywords = {}
    offset   = 0
    for tid, docs in cluster_docs.items():
        block    = tfidf[offset:offset+len(docs)]
        centroid = block.mean(axis=0).A1
        topn     = centroid.argsort()[-8:][::-1]
        keywords[tid] = [tfv.get_feature_names_out()[i] for i in topn]
        offset   += len(docs)

    # 6) Assemble report
    counts = df.groupby('topic_id').review.count()
    likes  = df[df.voted_up].groupby('topic_id').review.count()
    best   = (
        df.loc[df.groupby('topic_id').votes_up.idxmax(),
               ['topic_id','review']]
          .set_index('topic_id')
    )

    report = pd.DataFrame({
        'steam_appid':   appid,
        'Theme':         list(themes.keys()),
        '#Reviews':      counts.values,
        'LikeRatio':     (likes/counts*100).round(1).astype(str) + '%',
        'TopKeywords':   [", ".join(keywords[i]) for i in range(len(themes))],
        'ExampleReview': best.reindex(range(len(themes)))['review'].tolist()
    })
    return report


In [6]:
# Cell 6: Parallel dispatch of per‐game analysis

from dask import compute

# 1) Build one delayed task per game
tasks = [
    analyse_one_game(appid, themes)
    for appid, themes in GAME_THEMES.items()
]

# 2) Trigger all tasks in parallel on your Dask cluster
reports = compute(*tasks)   # runs each analyse_one_game concurrently

# 3) Filter out any None results (e.g. missing files)
reports = [r for r in reports if r is not None]

# 4) Concatenate into a master DataFrame
final_report = pd.concat(reports, ignore_index=True)

# 5) Persist & preview
final_report.to_csv('steam_theme_reports.csv', index=False)
display(final_report.head(20))


,steam_appid,Theme,#Reviews,LikeRatio,TopKeywords,ExampleReview
0,391540,story,7309,98.2%,"story, game, great, characters, good, great st...",Edit: (I played the neutral ending. Killed a d...
1,391540,characters,12025,97.0%,"10, 10 10, game, good, play, game 10, good gam...",[code]\n(The positive reviews...\nfill you wit...
2,391540,gameplay,4899,89.2%,"game, like, really, rpg, just, good, don, game...",[b] If you enjoy this review please consider f...
3,391540,combat,14177,95.9%,"game, good, play, good game, just, love, like,...","Have you ever noticed how, in a typical RPG (a..."
4,391540,music,6191,97.8%,"undertale, game, play, games, like, characters...",Ten dollars. Only a few hours of gameplay. 16-...
5,391540,visuals,4453,99.4%,"amazing, amazing game, game, game amazing, sou...",I tried. \n\nI'd been getting kind of weary of...
6,391540,humor_dialogue,19856,94.3%,"good, like, just, sans, funny, play, skeleton,...","for the first time in my life, i felt like i h..."
7,391540,morality,9201,96.0%,"game, great, great game, love, game great, lov...",this game made me love the two most hated font...
8,391540,secrets_meta,5261,98.1%,"fun, game, fun game, good, really, game fun, r...",---{Gameplay}---\n🔳Try not to get addicted\n☑️...
9,391540,warp,6831,98.7%,"best, played, best game, game, ve, games, ve p...",This is one of those few games that come along...
